# MsThesis draft

## Progress

### weeks 1-4 

Started from the end of the second week. Although, all the tools were ready.

**What**: neural ODEs, LMU 
**Why**: 
 - to represent data continuously
 - to solve problems with:
     - irregular sampling
     - low sampling rates
 - to lay groundwork for space-continuous models

**Begin-talk**


**The project goal**  
The motivation rises from the desire to get algorithms which is able to work with irregularly and rare sampled data from the real-world.  
**The main idea**  
We have continuous-in-depth LSTM's which allows to represent continuous-in-time signal. 
**The expected result**  
Obtain continuous-time representation of a signals. Check out what applications of the continuity of time there are.

Write:
1. Abstract
2. Introduction
3. Literature

For these see [the link](https://www.overleaf.com/read/rjvmxvkmgvyg)

    State your problem, write a description of your basic algorithm, prepare your computational experiment

    Basic code, draft report on the basic algorithm.

    Code, visual presentation of results

    Describe the algorithm

Cложный уровень: выставить гипотезы и условия. Сформулировать и доказать теорему.

    Make the error and quality analysis. Finalise the computational experiment. 

    Prepare the theoretical part and computational experiment. Explain the figures, write conclusions. Ready to the second checkpoint. 

    finalisation

## Code part

### Imports 

In [1]:
#import patch_path
from code_p300.datasets import P300Dataset, ListDataset
from code_p300.utils import data_dir
from code_p300 import visualizers as vis
import torch

ModuleNotFoundError: No module named 'code_p300'

### Data 

In [ ]:
# dataset collected at our lab
# it's the lightest one I know, so I'll use it
# in the upcoming weeks it will be public and available through moabb package
# also I will test everythong on public datasets like EPFL and BrainInvaders (through our interfaces)
# there's an option to work with moabb datasets directly. 
# But in my opinion, our version is more flexible ad familiar

In [ ]:
# Some records have different number of acts. 
# It doesn't affect anything, but raises warnings while creating dataset
# Will be fixed soon
import warnings
warnings.filterwarnings("ignore", category = UserWarning)

In [ ]:
ds_train = P300Dataset(data_dir / 'huge_demons', split = 'train')
ds_train

In [ ]:
ds_val = P300Dataset(data_dir / 'huge_demons', split = 'val')
ds_val

In [ ]:
binds_train = ds_train.binary_dataset()
binds_val = ds_val.binary_dataset()
binds_train, binds_val

In [ ]:
def binary_transform(bin_ds):
        '''Transforms data of binary dataset to friendly format, leaving only target epochs

        Args:
            bin_ds: BinaryDataset instance

        Returns:
            list of (epoch, label) tuple
        '''

        items = []
        for epochs_i, labels_i in bin_ds:
            for epoch, label in zip(epochs_i, labels_i):
                #if label==1:
                items.append((torch.flatten(epoch),label))
        return items


In [ ]:
data = binary_transform(binds_train)
train_set = ListDataset(data)

In [ ]:
data = binary_transform(binds_val)
valid_set = ListDataset(data)

### Simple use of AE as encoding (just to remember what it's like)

In [ ]:
from code_p300.bayesian import  vae

In [ ]:
from torch.utils.data import  DataLoader
from tqdm import tqdm
from torch import optim

def train_on_batch(model,
                   batch_of_x,
                   batch_of_y,
                   optimizer):

    model.zero_grad()
    loss = model.loss(batch_of_x, batch_of_y)
    loss.backward()
    optimizer.step()
    return


def train_epoch(model,
                train_generator,
                optimizer):

    model.train()
    for it, (batch_of_x, batch_of_y) in enumerate(train_generator):
        train_on_batch(model, batch_of_x, batch_of_y, optimizer)
    return


def trainer(model,
            optimizer,
            dataset,
            count_of_epoch=5,
            batch_size=64,
            progress=None):

    iterations = range(count_of_epoch)
    
    if progress is not None:
        iterations = progress(iterations)

    for it in iterations:
        batch_generator = DataLoader(
            dataset=dataset,
            batch_size=batch_size,
            shuffle=True,
            pin_memory=True)
        train_epoch(
            model=model,
            train_generator=batch_generator,
            optimizer=optimizer)
    return

In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [3]:
from code_p300.models import clfs_full, bin_scores

ModuleNotFoundError: No module named 'code_p300'

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression

def encode_epochs(model, bin_ds):
    '''Gets BinaryDataset and transforms it to encoder latent space images of epochs and labels

    Encodes epochs with `encoder` part of passed model

    Args:
        model: model with encoder part to get latent representation of epochs
        bin_ds: BinaryDataset to transform

    Returns:
        tuple of arrays of epochs and labels
    '''
    epochs = []
    labels = []
    for epochs_i, labels_i in bin_ds:
        distr = model.q_z(epochs_i.view(-1, 240))
        encoded = model.sample_z(distr)
        encoded = encoded.detach()
        epochs.append(torch.flatten(encoded, 1))
        labels.append(labels_i)
        
    return torch.cat(epochs).numpy(), np.array(labels)


def evaluate_ae(model, bin_train, bin_val):
    '''Calculates binary metrics on validation dataset

    Transforms dataset with encode_epochs and uses LogisticRegression for predictions

    Args:
        model: already trained autoencoder to use
        dataset_train: dataset to train LogisticRegression on
        dataset_val: dataset for evaluation

    Returns:
        dict of scores values: accuracy, f1_score and roc-auc
    '''

    encoded_train, labels_train = encode_epochs(model, bin_train)
    encoded_val, labels_val = encode_epochs(model, bin_val)

    lr = LogisticRegression(class_weight='balanced')
    lr.fit(encoded_train, labels_train)
    labels_pred = lr.predict(encoded_val)

    acc = accuracy_score(labels_val, labels_pred)
    f1 = f1_score(labels_val, labels_pred)
    roc_auc = roc_auc_score(labels_val, labels_pred)
    return {'accuracy': acc, 'f1_score': f1, 'roc-auc': roc_auc}


In [ ]:
from collections import defaultdict

from sklearn.base import clone

from code_p300.types import Directory
from code_p300.utils import score, tmp_dir


def evaluate_game(
    dataset: P300Dataset,
    train_acts: int,
    classifier,
    scores: tuple,
    *,
    augment_vae= None,
    vae = None,
    artifacts_dir: Directory = tmp_dir,
    verbose: bool = False,
    log: bool = True,
):
    '''Tests given dataset for metrics with default trainable classifier

    Args:
        dataset: obvious
        train_acts: number of first acts for each person to train on
        classifier: classifier object with standard sklearn interface

    Returns
        Dict with means and stds for each calculated metric
    '''
    # *** appears to be partly private code and depending on private constructs***
    # affects only multiclass labels, so must be ok not to use it

In [ ]:
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)
plt.rc('axes', titlesize=MEDIUM_SIZE)
plt.rc('axes', labelsize=MEDIUM_SIZE)
plt.rc('xtick', labelsize=SMALL_SIZE)
plt.rc('ytick', labelsize=SMALL_SIZE)
plt.rc('legend', fontsize=SMALL_SIZE)
plt.rc('figure', titlesize=BIGGER_SIZE)

In [ ]:
def plot_orig_reconstr(original_x, reconstructed_x):
    plt.figure(figsize = (14, 8))
    ticks = np.linspace(0, 30, 9)
    labels = np.linspace(ds_train.epoch_start*1000, ds_train.epoch_end*1000, 9, dtype=int)
    plt.plot(original_x.mean(axis=0), color = 'b', label = 'original eeg')
    plt.plot(reconstructed_x.mean(axis=0), color='g', label = 'reconstructed eeg')
    plt.xticks(ticks = ticks, labels = labels)
    plt.xlabel('miliseconds')
    plt.ylabel('normalized mkv')
    plt.grid(True)
    plt.legend(loc='best')
    plt.show()

In [ ]:
def binary_transform_target(bin_ds):
        '''Transforms data of binary dataset to friendly format, leaving only target or non-target epochs

        Args:
            bin_ds: BinaryDataset instance

        Returns:
            list of (epoch, label) tuple
        '''

        items = []
        for epochs_i, labels_i in bin_ds:
            for epoch, label in zip(epochs_i, labels_i):
                if (label.numpy()==1):
                    items.append((torch.flatten(epoch),label))
        return items


In [ ]:
def plot_orig_sampled(original_x, reconstructed_x):
    plt.figure(figsize = (14, 8))
    ticks = np.linspace(0, 30, 9)
    labels = np.linspace(ds_train.epoch_start*1000, ds_train.epoch_end*1000, 9, dtype=int)
    plt.plot(original_x.mean(axis=0), color = 'b', label = 'original eeg')
    plt.plot(reconstructed_x.mean(axis=0), color='g', label = 'sampled eeg')
    plt.xticks(ticks = ticks, labels = labels)
    plt.xlabel('miliseconds')
    plt.ylabel('normalized mkv')
    plt.grid(True)
    plt.legend(loc='best')
    plt.show()

In [ ]:
epoch_shape= binds_train[0][0].shape[1:]
epoch_shape

#### train model 

In [ ]:
model = vae.VAE(32, epoch_shape[0]*epoch_shape[1], device='cpu')

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
trainer(model=model,
        optimizer=optimizer,
        dataset=train_set,
        count_of_epoch=30,
        batch_size=1024,
        progress=tqdm)

#### evaluate model

In [ ]:
evaluate_ae(model, train_set, valid_set)

In [ ]:
benchmark = evaluate_game(ds_train, 5, clfs_full['LR'][0], bin_scores, vae = model, log=False)
acc_mean = np.mean(benchmark['mult']['accuracy'])
acc_std = np.std(benchmark['mult']['accuracy'])
acc_median = np.median(benchmark['mult']['accuracy'])
print('train mult_accuracy:',acc_mean.round(3), acc_std.round(3), acc_median.round(3))

benchmark = evaluate_game(ds_val, 5, clfs_full['LR'][0], bin_scores, vae = model, log=False)
acc_mean = np.mean(benchmark['mult']['accuracy'])
acc_std = np.std(benchmark['mult']['accuracy'])
acc_median = np.median(benchmark['mult']['accuracy'])
print('val mult_accuracy:',acc_mean.round(3), acc_std.round(3), acc_median.round(3))

In [ ]:
qz = model.q_z(torch.stack([valid_set[0][0], valid_set[1][0], valid_set[9][0]]))


In [ ]:
qzsample = model.sample_z(qz)

In [ ]:
model_x = model.q_x(qzsample)[:, 0, :]


In [ ]:
original_x = valid_set[9][0].view((8,30)).cpu().data.numpy()
reconstructed_x = model_x[2].view((8,30)).cpu().data.numpy()

In [ ]:
plot_orig_reconstr(original_x, reconstructed_x)

In [ ]:
original_x = valid_set[1][0].view((8,30)).cpu().data.numpy()
reconstructed_x = model_x[1].view((8,30)).cpu().data.numpy()

plot_orig_reconstr(original_x, reconstructed_x)

### Augmentations

#### Target/non-target data

In [ ]:
data = binary_transform_target(binds_train)
train_set = ListDataset(data)

In [ ]:
data = binary_transform_target(binds_val)
valid_set = ListDataset(data)

In [ ]:
alltargets_train = ListDataset(binary_transform(binds_train))
alltargets_val = ListDataset(binary_transform(binds_val))

In [ ]:
benchmark = evaluate_game(ds_train, 5, clfs_full['LR'][0], bin_scores, log=False)
acc_mean = np.mean(benchmark['mult']['accuracy'])
acc_std = np.std(benchmark['mult']['accuracy'])
acc_median = np.median(benchmark['mult']['accuracy'])
print('train mult_accuracy:',acc_mean.round(3), acc_std.round(3), acc_median.round(3))
roc_auc = np.mean(benchmark['bin']['roc_auc'])
f1 = np.mean(benchmark['bin']['f1'])
print('train roc_auc:', roc_auc.round(3))
print('train f1' , f1.round(3))

benchmark = evaluate_game(ds_val, 5, clfs_full['LR'][0], bin_scores,  log=False)
acc_mean = np.mean(benchmark['mult']['accuracy'])
acc_std = np.std(benchmark['mult']['accuracy'])
acc_median = np.median(benchmark['mult']['accuracy'])
print('val mult_accuracy:',acc_mean.round(3), acc_std.round(3), acc_median.round(3))
roc_auc = np.mean(benchmark['bin']['roc_auc'])
f1 = np.mean(benchmark['bin']['f1'])
print('val roc_auc:', roc_auc.round(3))
print('val f1' , f1.round(3))

In [ ]:
model = vae.VAE(64, epoch_shape[0]*epoch_shape[1], device='cpu')

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
trainer(model=model,
        optimizer=optimizer,
        dataset=train_set,
        count_of_epoch=100,
        batch_size=1024,
        progress=tqdm)

##### Trained only on targets

In [ ]:
benchmark = evaluate_game(ds_train, 5, clfs_full['LR'][0], bin_scores, augment_vae = model, log=False)
acc_mean = np.mean(benchmark['mult']['accuracy'])
acc_std = np.std(benchmark['mult']['accuracy'])
acc_median = np.median(benchmark['mult']['accuracy'])
print('train mult_accuracy:',acc_mean.round(3), acc_std.round(3), acc_median.round(3))
roc_auc = np.mean(benchmark['bin']['roc_auc'])
f1 = np.mean(benchmark['bin']['f1'])
print('train roc_auc:',roc_auc.round(3))
print('train f1' , f1.round(3))

benchmark = evaluate_game(ds_val, 5, clfs_full['LR'][0], bin_scores,augment_vae = model,  log=False)
acc_mean = np.mean(benchmark['mult']['accuracy'])
acc_std = np.std(benchmark['mult']['accuracy'])
acc_median = np.median(benchmark['mult']['accuracy'])
print('val mult_accuracy:',acc_mean.round(3), acc_std.round(3), acc_median.round(3))
roc_auc = np.mean(benchmark['bin']['roc_auc'])
f1 = np.mean(benchmark['bin']['f1'])
print('val roc_auc:', roc_auc.round(3))
print('val f1' , f1.round(3))

In [ ]:
sample = model.generate_samples(100).mean(dim=0)

In [ ]:
original_x = torch.stack([valid_set[i][0] for i in range(len(valid_set))]).mean(dim=0)
original_x = original_x.view((8,30)).cpu().data.numpy()
reconstructed_x = sample.view((8,30)).cpu().data.numpy()

In [ ]:
plot_orig_sampled(original_x, reconstructed_x)

##### Trained on all dataset

In [ ]:
benchmark = evaluate_game(ds_train, 5, clfs_full['LR'][0], bin_scores, augment_vae = model, log=False)
acc_mean = np.mean(benchmark['mult']['accuracy'])
acc_std = np.std(benchmark['mult']['accuracy'])
acc_median = np.median(benchmark['mult']['accuracy'])
print('train mult_accuracy:',acc_mean.round(3), acc_std.round(3), acc_median.round(3))
roc_auc = np.mean(benchmark['bin']['roc_auc'])
f1 = np.mean(benchmark['bin']['f1'])
print('train roc_auc:',roc_auc.round(3))
print('train f1' , f1.round(3))

benchmark = evaluate_game(ds_val, 5, clfs_full['LR'][0], bin_scores,augment_vae = model,  log=False)
acc_mean = np.mean(benchmark['mult']['accuracy'])
acc_std = np.std(benchmark['mult']['accuracy'])
acc_median = np.median(benchmark['mult']['accuracy'])
print('val mult_accuracy:',acc_mean.round(3), acc_std.round(3), acc_median.round(3))
roc_auc = np.mean(benchmark['bin']['roc_auc'])
f1 = np.mean(benchmark['bin']['f1'])
print('val roc_auc:', roc_auc.round(3))
print('val f1' , f1.round(3))

In [ ]:
sample = model.generate_samples(100).mean(dim=0)

In [ ]:
original_x = torch.stack([valid_set[i][0] for i in range(len(valid_set))]).mean(dim=0)
original_x = original_x.view((8,30)).cpu().data.numpy()
reconstructed_x = sample.view((8,30)).cpu().data.numpy()

In [ ]:
plot_orig_reconstr(original_x, reconstructed_x)

In [ ]:
import code

#  !

In [ ]:
from code.datasets import get_data
from code.models import NeuralCDE, ODELSTM, LMUCell
import torchcde
import torch


In [ ]:
LMUCell(input_size = 3, hidden_size = 5, memory_size = 2, theta = 0.5)

In [ ]:
num_epochs=30
train_X, train_y = get_data()

######################
# input_channels=3 because we have both the horizontal and vertical position of a point in the spiral, and time.
# hidden_channels=8 is the number of hidden channels for the evolving z_t, which we get to choose.
# output_channels=1 because we're doing binary classification.
######################
model = NeuralCDE(input_channels=3, hidden_channels=8, output_channels=1)
optimizer = torch.optim.Adam(model.parameters())

######################
# Now we turn our dataset into a continuous path. We do this here via natural cubic spline interpolation.
# The resulting `train_coeffs` is a tensor describing the path.
# For most problems, it's probably easiest to save this tensor and treat it as the dataset.
######################
train_coeffs = torchcde.natural_cubic_coeffs(train_X)

train_dataset = torch.utils.data.TensorDataset(train_coeffs, train_y)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32)
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch_coeffs, batch_y = batch
        pred_y = model(batch_coeffs).squeeze(-1)
        loss = torch.nn.functional.binary_cross_entropy_with_logits(pred_y, batch_y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print('Epoch: {}   Training loss: {}'.format(epoch, loss.item()))

test_X, test_y = get_data()
test_coeffs = torchcde.natural_cubic_coeffs(test_X)
pred_y = model(test_coeffs).squeeze(-1)
binary_prediction = (torch.sigmoid(pred_y) > 0.5).to(test_y.dtype)
prediction_matches = (binary_prediction == test_y).to(test_y.dtype)
proportion_correct = prediction_matches.sum() / test_y.size(0)
print('Test Accuracy: {}'.format(proportion_correct))


In [ ]:
train_X, train_y = get_data()
######################
# input_channels=3 because we have both the horizontal and vertical position of a point in the spiral, 
# and time.
# hidden_channels=8 is the number of hidden channels for the evolving z_t, which we get to choose.
# output_channels=1 because we're doing binary classification.
######################
model = ODELSTM(in_features=2, hidden_size=8, out_feature=1, return_sequences=False)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

######################
# Now we turn our dataset into a continuous path. We do this here via natural cubic spline interpolation.
# The resulting `train_coeffs` are some tensors describing the path.
# For most problems, it's advisable to save these coeffs and treat them as the dataset, 
# as this interpolation can take a long time.
######################
#train_coeffs = controldiffeq.natural_cubic_spline_coeffs(train_t, train_X)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma=0.5)
train_dataset = torch.utils.data.TensorDataset(train_X, train_y)
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32)
for epoch in range(30):
    for batch in train_dataloader:
        batch_x, batch_y = batch
        pred_y = model(batch_x[:, :, 1:], batch_x[:, :, 0]).squeeze(-1)
        loss = torch.nn.functional.binary_cross_entropy_with_logits(
            pred_y, batch_y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    scheduler.step()
    print('Epoch: {}   Training loss: {}    lr: {}'.format(epoch, loss.item(), optimizer.param_groups[0]['lr']))




# Old stuff 